# Парсинг данных. Практика 3

In [70]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
from tqdm import tqdm

## Задание

Дана ссылка на первую страницу [каталога книжных новинок](https://www.bgshop.ru/catalog/group?id=444&page=1&sort=1&instock=). Соберите информацию о книгах со всех страниц. 

In [2]:
urls = []

for i in range(1, 3):
    urls.append(f'https://www.bgshop.ru/catalog/group?id=444&page={i}&sort=1&instock=')
urls

['https://www.bgshop.ru/catalog/group?id=444&page=1&sort=1&instock=',
 'https://www.bgshop.ru/catalog/group?id=444&page=2&sort=1&instock=']

In [3]:
page = requests.get(urls[0])
page.encoding='utf-8'

In [4]:
soup = BeautifulSoup(page.text)
soup

<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="fd9a658378927472" name="mailru-verification"/>
<title>Книжные новинки | Библио-Глобус</title>
<meta content="Еженедельное обновление новой литературы от крупных издательств. Будь в курсе самых популярных книжных новинок!" name="description"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="all,follow" name="robots"/>
<link href="/favicon.png" rel="icon" type="image/png"/>
<link href="/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="/searchprovider.xml" rel="search" title="Biblio-Globus" type="application/opensearchdescription+xml"/>
<link href="https://static1.bgshop.ru" rel="dns-prefetch"/>
<link href="https://static2.bgshop.ru" rel="dns-prefetch"/>
<link href="https://fonts.googleapis.com" rel="dns-prefetch"/>
<link href="https://www.google.com" rel="dns-prefetch"/>
<link href="https:/

In [5]:
product = soup.find_all('div', {'class':'product'})[0]
product

<div class="product">
<a class="img_link" href="/product/10955592"><img alt="" class="img-fluid" src="https://static1.bgshop.ru/imagehandler.ashx?fileName=10955592.jpg&amp;width=200"/></a>
<div class="text">
<div class="author">Хабенский  К.Ю.</div>
<h3><a href="/product/10955592" id="p_title_10955592">Московский художественный театр. МХТ 125 лет на сцене</a></h3>
<div class="prices_item">
<div class="price_item_wrapper"><span class="price_item_title">Цена на сайте:</span> <div class="price_item_block"><s>1939 ₽</s> <span class="price_item_new price_item_with_discount">1745 ₽</span></div></div>
<div class="price_item_wrapper bt-divider"><span class="price_item_title">Цена в магазине:</span> <div class="price_item_block"><span class="price_item_new">1939 ₽</span></div></div>
</div>
<p class="status im_status_title"><span style="color: #3ba155;"> в наличии</span><span class="product-qty">19 шт.</span></p>
<div class="buttons row">
<div class="col-9 pl-0">
<span class="btn btn-block btn-p

In [24]:
'https://www.bgshop.ru' + product.find_all('a')[0].get('href')

'https://www.bgshop.ru/product/10955592'

In [11]:
product.find('div', class_="author").text

'Хабенский  К.Ю.'

In [21]:
product.find('a', {'id': re.compile(r'.+')}).text

'Московский художественный театр. МХТ 125 лет на сцене'

In [22]:
product.find('span', class_="price_item_new price_item_with_discount").text

'1745\xa0₽'

In [26]:
# но не везде цены со скидками
soup.find_all('div', {'class':'product'})[3].find('span', class_="price_item_new price_item_with_discount").text

AttributeError: 'NoneType' object has no attribute 'text'

In [27]:
soup.find_all('div', {'class':'product'})[3].find('span', class_="price_item_new").text

'1909\xa0₽'

In [33]:
price = product.find('span', class_="price_item_new price_item_with_discount").text

In [65]:
# сразу научимся чистить цену

price_clean = int(re.findall(r'\d+', price)[0])
price_clean

1745

In [66]:
qty = soup.find('span', class_='product-qty').text

In [69]:
qty_clean = int(re.findall(r'\d+', qty)[0])
qty_clean

19

Спарсим все данные.

In [95]:
books = []

for u in tqdm(urls):
    page = requests.get(u)
    soup = BeautifulSoup(page.text)
    sleep(1)
    info = soup.find_all('div', {'class':'product'})
    for i in info:
        author = i.find('div', class_="author").text
        title = i.find('a', {'id': re.compile(r'.+')}).text
        try:
            price = i.find('span', class_="price_item_new price_item_with_discount").text            
        except IndexError:
            price = i.find('span', class_="price_item_new").text
        except:
            price = i.find('span', class_="price_item_new_lg").text
            
        price_clean = int(re.findall(r'\d+', price)[0])    
        qty = soup.find('span', class_='product-qty').text
        qty_clean = int(re.findall(r'\d+', qty)[0])
        url0 = 'https://www.bgshop.ru' + i.find('a').get('href')

        books.append((author, title, price_clean, qty_clean, url0))

100%|█████████████████████████████████████████████| 2/2 [00:03<00:00,  1.84s/it]


In [103]:
df = pd.DataFrame(books)
df.columns = ['Автор', 'Название', 'Цена', 'Количество', 'Ссылка']

In [104]:
df

,Автор,Название,Цена,Количество,Ссылка
0,Хабенский К.Ю.,Московский художественный театр. МХТ 125 лет н...,1745,19,https://www.bgshop.ru/product/10955592
1,"Пушкин А.С., Гоголь Н.В.,Чехов А.П., Погорельс...",Русская классика в иллюстрациях Геннадия Спирина,1224,19,https://www.bgshop.ru/product/10955184
2,Mcconaughey Matthew,Greenlights,1909,19,https://www.bgshop.ru/product/10954702
3,Galbraith Robert,Ink black heart,1909,19,https://www.bgshop.ru/product/10954697
4,"Цыбулько И. П, Дощинский Р. А.",ЕГЭ-2024. Русский язык: типовые экзаменационны...,854,19,https://www.bgshop.ru/product/10954676
5,Биркби Р.,Горное безумие. Биография Скотта Фишера,989,19,https://www.bgshop.ru/product/10954574
6,,Ленинград. Самые знаковые здания и места город...,1539,19,https://www.bgshop.ru/product/10953869
7,"Друэ В., Вьель П.-Л.",Праздник как по нотам. Закуски: 80 оригинальны...,645,19,https://www.bgshop.ru/product/10952285
8,Е. Манойло,Ветер уносит мертвые листья,654,19,https://www.bgshop.ru/product/10951222
9,Мисима Ю.,После банкета,664,19,https://www.bgshop.ru/product/10949732


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Автор       19 non-null     object
 1   Название    19 non-null     object
 2   Цена        19 non-null     int64 
 3   Количество  19 non-null     int64 
 4   Ссылка      19 non-null     object
dtypes: int64(2), object(3)
memory usage: 888.0+ bytes


In [106]:
df = df.sort_values('Цена', ascending = False).reset_index(drop=True)

In [107]:
df

,Автор,Название,Цена,Количество,Ссылка
0,В. Ложкин,"Доброе утро, Котоград!",2532,3,https://www.bgshop.ru/product/10948842
1,А. П. Ситников,KARMACOACH,1979,19,https://www.bgshop.ru/product/10949710
2,Mcconaughey Matthew,Greenlights,1909,19,https://www.bgshop.ru/product/10954702
3,Galbraith Robert,Ink black heart,1909,19,https://www.bgshop.ru/product/10954697
4,Хабенский К.Ю.,Московский художественный театр. МХТ 125 лет н...,1745,19,https://www.bgshop.ru/product/10955592
5,,Ленинград. Самые знаковые здания и места город...,1539,19,https://www.bgshop.ru/product/10953869
6,П.В. Тулаев,Российский Крым. Обзор большой истории,1538,3,https://www.bgshop.ru/product/10948177
7,Наумчик А.Е.,Кимчи. Символ корейской кухни.,1338,3,https://www.bgshop.ru/product/10947369
8,Кобзон И,Иосиф Кобзон. Мгновения...,1289,3,https://www.bgshop.ru/product/10943909
9,Браунштайн М.,Техники сухого рисования,1224,3,https://www.bgshop.ru/product/10947798


In [82]:
df.to_excel('bg_books.xlsx')

## Задание 2

По аналогии спарсите все книги, в которых есть слово "Python".

In [ ]:
# ваш код